In [43]:
# pyg-team
# from: https://github.com/pyg-team/pytorch_geometric/blob/master/examples/link_pred.py

In [54]:

import os.path as osp

import torch
from sklearn.metrics import roc_auc_score

import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
from torch_geometric.nn import GCNConv
from torch_geometric.utils import negative_sampling

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
transform = T.Compose([
    T.NormalizeFeatures(),
    T.ToDevice(device),
    T.RandomLinkSplit(num_val=0.05, num_test=0.1, is_undirected=True,
                      add_negative_train_samples=False),
])
# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
path = 'Planetoid'
dataset = Planetoid(path, name='Cora', transform=transform)
dataset[0]
# After applying the `RandomLinkSplit` transform, the data is transformed from
# a data object to a list of tuples (train_data, val_data, test_data), with
# each element representing the corresponding split.

# train_data, val_data, test_data = dataset[0]


(Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[4488], edge_label_index=[2, 4488]),
 Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[526], edge_label_index=[2, 526]),
 Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054]))

In [53]:
dataset[0]

tensor([3, 4, 4,  ..., 3, 3, 3])

In [42]:
dataset[0]

(Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[4488], edge_label_index=[2, 4488]),
 Data(x=[2708, 1433], edge_index=[2, 8976], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[526], edge_label_index=[2, 526]),
 Data(x=[2708, 1433], edge_index=[2, 9502], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[1054], edge_label_index=[2, 1054]))

tensor(527., device='cuda:0')

In [17]:
test_data.edge_label_index
test_data.edge_index

tensor([[1998, 2326],
        [1998,  109],
        [1998,  695],
        [1998,  191],
        [1998, 1781],
        [1998, 1986]], device='cuda:0')

In [35]:
train_data.edge_label_index.shape

torch.Size([2, 3696])

In [37]:
train_data

Data(x=[2708, 1433], edge_index=[2, 7392], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708], edge_label=[3696], edge_label_index=[2, 3696])

In [41]:
# x
(train_data.x[0] > 0).sum()

tensor(9, device='cuda:0')

In [42]:
# edge_index
train_data.edge_label_index.T.shape

torch.Size([4488, 2])

In [43]:
dataset.num_features

1433

In [44]:
train_data.edge_label

tensor([1., 1., 1.,  ..., 1., 1., 1.], device='cuda:0')

In [49]:
dataset.num_features

1433

In [35]:
class Net(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def encode(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        return self.conv2(x, edge_index)

    def decode(self, z, edge_label_index):
        return (z[edge_label_index[0]] * z[edge_label_index[1]]).sum(dim=-1)

    def decode_all(self, z):
        prob_adj = z @ z.t()
        return (prob_adj > 0).nonzero(as_tuple=False).t()


model = Net(dataset.num_features, 128, 64).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()


def train():
    model.train()
    optimizer.zero_grad()
    z = model.encode(train_data.x, train_data.edge_index)

    # We perform a new round of negative sampling for every training epoch:
    neg_edge_index = negative_sampling(
        edge_index=train_data.edge_index, num_nodes=train_data.num_nodes,
        num_neg_samples=train_data.edge_label_index.size(1), method='sparse')

    edge_label_index = torch.cat(
        [train_data.edge_label_index, neg_edge_index],
        dim=-1,
    )
    edge_label = torch.cat([
        train_data.edge_label,
        train_data.edge_label.new_zeros(neg_edge_index.size(1))
    ], dim=0)

    out = model.decode(z, edge_label_index).view(-1)
    loss = criterion(out, edge_label)
    loss.backward()
    optimizer.step()
    return loss


@torch.no_grad()
def test(data):
    model.eval()
    z = model.encode(data.x, data.edge_index)
    out = model.decode(z, data.edge_label_index).view(-1).sigmoid()
    return roc_auc_score(data.edge_label.cpu().numpy(), out.cpu().numpy())


best_val_auc = final_test_auc = 0
for epoch in range(1, 101):
    loss = train()
    val_auc = test(val_data)
    test_auc = test(test_data)
    if val_auc > best_val_auc:
        best_val_auc = val_auc
        final_test_auc = test_auc
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Val: {val_auc:.4f}, '
          f'Test: {test_auc:.4f}')

print(f'Final Test: {final_test_auc:.4f}')

z = model.encode(test_data.x, test_data.edge_index)
final_edge_index = model.decode_all(z)

Epoch: 001, Loss: 0.6930, Val: 0.6182, Test: 0.7072
Epoch: 002, Loss: 0.6814, Val: 0.6115, Test: 0.7072
Epoch: 003, Loss: 0.7185, Val: 0.6099, Test: 0.7150
Epoch: 004, Loss: 0.6768, Val: 0.6180, Test: 0.7298
Epoch: 005, Loss: 0.6855, Val: 0.6380, Test: 0.7387
Epoch: 006, Loss: 0.6895, Val: 0.6533, Test: 0.7425
Epoch: 007, Loss: 0.6908, Val: 0.6468, Test: 0.7295
Epoch: 008, Loss: 0.6910, Val: 0.6335, Test: 0.7166
Epoch: 009, Loss: 0.6904, Val: 0.6262, Test: 0.7133
Epoch: 010, Loss: 0.6888, Val: 0.6221, Test: 0.7146
Epoch: 011, Loss: 0.6858, Val: 0.6174, Test: 0.7139
Epoch: 012, Loss: 0.6811, Val: 0.6097, Test: 0.7119
Epoch: 013, Loss: 0.6774, Val: 0.6046, Test: 0.7095
Epoch: 014, Loss: 0.6788, Val: 0.6022, Test: 0.7055
Epoch: 015, Loss: 0.6764, Val: 0.6063, Test: 0.7028
Epoch: 016, Loss: 0.6697, Val: 0.6196, Test: 0.7037
Epoch: 017, Loss: 0.6650, Val: 0.6424, Test: 0.7113
Epoch: 018, Loss: 0.6610, Val: 0.6591, Test: 0.7158
Epoch: 019, Loss: 0.6551, Val: 0.6644, Test: 0.7123
Epoch: 020, 